In [1]:
# Imports
import torch.nn as nn
import torch

In [2]:
# Task 0. Generator
class Generator(nn.Module):
    """
    Generator subclass for the generator side of the network
    """
    def __init__(self,
                 input_size,
                 hidden_size,
                 output_size):
        """
        Initialize Generator class

        Args:
            input_size - integer size of the input tensor
            hidden_size - integer size of the hidden layer
            output_size - integer size of the output layer
        """
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        """
        Forward pass of the Generator network

        Args:
            x - Input tensor of shape (batch_size, input_size)

        Returns:
            Output tensor of shape (batch_size, output_size)
        """
        return self.main(x)


In [3]:
# 0-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
genTester = Generator

G_Test = genTester(1,1,1)

print(G_Test)

Generator(
  (main): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1, out_features=1, bias=True)
    (3): Tanh()
    (4): Linear(in_features=1, out_features=1, bias=True)
  )
)


In [4]:
# Task 1. Discriminator
class Discriminator(nn.Module):
    """
    Discriminator subclass for the discriminator side of the network
    """
    def __init__(self,
                 input_size,
                 hidden_size,
                 output_size):
        """
                Initialize Discriminator class

        Args:
            input_size - integer size of the input tensor
            hidden_size - integer size of the hidden layer
            output_size - integer size of the output layer
        """
        super().__init__()

        self.main = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Sigmoid(),
            nn.Linear(hidden_size, hidden_size),
            nn.Sigmoid(),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        """
        Forward pass of the Discriminator network

        Args:
            x - Input tensor of shape (batch_size, input_size)

        Returns:
            Output tensor of shape (batch_size, output_size)
        """
        return self.main(x)

In [5]:
# 1-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

disTester = Discriminator

D_Test = disTester(1,1,1)

print(D_Test)

Discriminator(
  (main): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1, out_features=1, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=1, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [6]:
# Task 2. Sample Z
def sample_Z(mu, sigma, sampleType, size=(1, 1)):
    """
    Creates input for the generator and discriminator

    Added size argument to sample_Z function. Per torch.randn and torch.normal
        documentataion, size is a sequence of integers defining the shape of
        the output tensor.

    Args:
        mu -  The mean of the distribution
        sigma - The standard deviation of the distribution
        sampleType - A variable that selects which model to sample for
            The variable should accept a G or D as string values
        size - tuple of ints defining the shape of the output tensor

    Returns:
        a torch.Tensor type for both generator and discriminator if the
            parameters are correct, otherwise return 0
    """
    # For the generator, find a random number from torch.rand
    # Multiply that number by sigma and add mu
    # From the task, input data for the generator should be random
    if sampleType == "G":
        return torch.randn(size=size)
    # For the discriminator, take a number from the normal distribution
    # using torch.normal
    elif sampleType == "D":
        return torch.normal(mean=mu, std=sigma, size=size)
    else:
        return 0


In [7]:
# 2-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

samplerTest = sample_Z

print(samplerTest(0,1,"D",(4,8)))

print(samplerTest(0,1,"G",(4,8)))

print(torch.normal(mean=0, std=1, size=(4, 8)))

print(samplerTest(0,1,"F",))

tensor([[ 0.5486,  0.1243,  0.7660,  1.1011, -0.2998,  0.9035, -0.6614,  1.0365],
        [-1.4639, -1.6256,  0.4025, -1.1328, -0.7011,  1.1065, -2.4644,  0.6711],
        [-1.4675,  1.5334,  1.4555, -0.8965, -0.0494, -0.6283, -0.6411,  0.9566],
        [-1.4363,  0.3333,  0.3031,  2.6562, -0.0749, -0.9279, -0.6393, -1.1496]])
tensor([[ 0.8919,  0.6002,  0.7805,  0.4731, -0.5750, -1.7568,  0.9302,  0.9657],
        [-0.5271, -1.0853,  0.6912, -0.0855, -0.4524,  0.9513, -0.2468, -0.2807],
        [ 0.8354,  1.7799,  0.9956, -0.2852,  0.8513,  0.9798,  0.0456, -0.7581],
        [ 0.3019,  1.4604, -0.7738,  0.9191,  1.8334, -2.0303, -0.0104, -2.9034]])
tensor([[-1.0152,  1.5736, -0.8049, -1.3343,  0.3935, -0.6195, -0.0503,  2.2336],
        [-0.5042,  0.0905,  1.1181, -0.1343, -0.6981, -0.7886, -0.8403,  0.2697],
        [-0.3926, -1.6843,  0.3854,  0.9726, -0.8232,  1.4888,  2.1650, -0.7809],
        [-0.5336,  0.4488,  0.7027, -0.1025,  1.2249,  0.8983, -0.4583,  1.6748]])
0


In [8]:
# Task 3. Train Discriminator
"""
Function to train the Discriminator
"""
import torch


def train_dis(Gen,
              Dis,
              dInputSize,
              gInputSize,
              mbatchSize,
              steps,
              optimizer,
              crit):
    """
    Args:
        Gen, Dis - Generator and Discriminator objects
        dInputSize - Input size of Discriminator input data
        gInputSize - Input size of Generator input data
        mbatchSize - Batch size for training
        steps - number of steps for training
        optimizer - stochastic gradient descent optimizer object
        crit - BCEloss function

    Should use both random noise and normal distribution for sampling
    The 4 moments should be used in processing the sample

    Returns:
        The function should return the error estimate of the fake and real data
        along with the fake and real data sets of type torch.Tensor()
    """
    for _ in range(steps):
        # Create real and generated data and labels to train discriminator

        # Use sample_Z function to make some real data points
        reals = sample_Z(mu=0.0,
                         sigma=1.0,
                         sampleType="D",
                         size=(mbatchSize, dInputSize))
        # Make labels for real data, equal to 1
        realLabels = torch.ones((mbatchSize, 1))

        # Use sample_Z function to make some generated data
        generated = sample_Z(mu=0.0,
                             sigma=1.0,
                             sampleType="G",
                             size=(mbatchSize, gInputSize))
        generatedOutput = Gen(generated)
        # Make labels for generated data, equal to 0
        generatedLabels = torch.zeros((mbatchSize, 1))

        # Combine real and generated data/labels to run through discriminator
        data = torch.cat((reals, generated))
        labels = torch.cat((realLabels, generatedLabels))

        # Train the Discriminator on combined data and labels
        Dis.zero_grad()
        output = Dis(data)
        loss = crit(output, labels)
        loss.backward()
        optimizer.step()

    discriminatorOut = Dis(data)
    discriminatorLoss = crit(discriminatorOut, labels)
    generatorOut = Dis(generatedOutput)
    generatorLoss = crit(generatorOut, generatedLabels)

    return generatorLoss, discriminatorLoss, data

In [9]:
# 3-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

disTrain = train_dis

learning_rate = 1e-3
sgd_momentum = 0.9
g_input_size = 1   
g_hidden_size = 5  
g_output_size = 1  
minibatch_size =  500 
d_input_size = 1 
d_hidden_size = 10 
d_output_size = 1 
num_epochs = 50
d_steps = 20
g_steps = 20

G_Test = Generator(1,5,1)
D_Test = Discriminator(1,10,1)



opt_Test = optim.SGD(G_Test.parameters(), lr=learning_rate, momentum=sgd_momentum)
crit_Test = nn.BCELoss() 

print(disTrain(G_Test,D_Test, d_input_size,g_input_size, minibatch_size, d_steps, opt_Test, crit_Test))

(tensor(0.7551, grad_fn=<BinaryCrossEntropyBackward0>), tensor(0.6950, grad_fn=<BinaryCrossEntropyBackward0>), tensor([[-1.1961e+00],
        [ 1.2884e+00],
        [-1.1794e-01],
        [ 3.9764e-01],
        [ 3.5494e-01],
        [-3.9188e-01],
        [ 1.5253e+00],
        [ 4.3318e-01],
        [-1.9738e+00],
        [-8.0660e-02],
        [-4.1774e-01],
        [ 1.2045e-01],
        [-1.6196e-01],
        [ 1.4117e-01],
        [ 3.5370e-03],
        [-3.9461e-01],
        [-9.0713e-01],
        [-1.2764e+00],
        [ 2.1315e+00],
        [ 1.1395e+00],
        [ 2.1444e+00],
        [-3.5879e-01],
        [-1.2199e+00],
        [ 4.9906e-01],
        [ 1.1866e+00],
        [-1.0364e+00],
        [-7.7984e-01],
        [ 1.1640e+00],
        [-1.7907e+00],
        [ 7.8938e-01],
        [-1.0430e+00],
        [ 3.6471e-01],
        [ 6.5868e-01],
        [ 1.6492e+00],
        [-1.2840e-01],
        [-4.1828e-01],
        [-3.6421e-01],
        [ 1.1741e-01],
        [-7.774

In [10]:
# Task 4. Train Generator
"""
Function to train the generator
"""
def train_gen(Gen,
              Dis,
              gInputSize,
              mbatchSize,
              steps,
              optimizer,
              crit):
    """
        Args:
            Gen, Dis - Generator and Discriminator objects
            gInputSize - Input size of Generator input data
            mbatchSize - Batch size for training
            steps - number of steps for training
            optimizer - stochastic gradient descent optimizer object
            crit - BCEloss function

        Should use only random noise for sampling
        The 4 moments should be used in processing the sample

        Returns:
            The function should return the error of the fake data and the fake
            data set of type torch.Tensor()
    """
    for _ in range(steps):
        # Generate data, same as discriminator training
        generated = sample_Z(mu=0.0,
                             sigma=1.0,
                             sampleType="G",
                             size=(mbatchSize, gInputSize))
        generatedOutput = Gen(generated)
        labels = torch.ones((mbatchSize, 1))

        # Train the generator
        Gen.zero_grad()
        output = Dis(generatedOutput)
        loss = crit(output, labels)
        loss.backward()
        optimizer.step()

    return loss, generated


In [11]:
# 4-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

genTrain = train_gen

learning_rate = 1e-3
sgd_momentum = 0.9
g_input_size = 1   
g_hidden_size = 5  
g_output_size = 4  
minibatch_size =  500 
d_input_size = 4 
d_hidden_size = 10 
d_output_size = 1  
num_epochs = 50
d_steps = 20
g_steps = 20



G_Test = Generator(1,5,4)
D_Test = Discriminator(4,10,1)



dOpt_Test = optim.SGD(D_Test.parameters(), lr=learning_rate, momentum=sgd_momentum)
crit_Test = nn.BCELoss()  


print(genTrain(G_Test,D_Test,g_input_size, minibatch_size, g_steps, dOpt_Test, crit_Test))

(tensor(0.2004, grad_fn=<BinaryCrossEntropyBackward0>), tensor([[ 2.5385e-01],
        [ 1.6575e+00],
        [ 1.3071e-01],
        [-1.3089e+00],
        [ 4.2240e-01],
        [-1.5861e+00],
        [ 6.4359e-01],
        [-6.9679e-01],
        [-6.8552e-01],
        [-8.1193e-01],
        [-5.6959e-01],
        [-5.6168e-01],
        [ 1.4386e+00],
        [-7.2997e-02],
        [ 2.1716e-01],
        [-1.2545e+00],
        [ 1.1778e+00],
        [-2.5636e+00],
        [-1.0999e+00],
        [ 6.5060e-01],
        [ 7.2390e-01],
        [-8.7635e-01],
        [-7.8108e-01],
        [-7.8372e-01],
        [ 9.3288e-01],
        [ 6.1078e-01],
        [ 5.2377e-01],
        [ 5.3597e-01],
        [-2.8390e-01],
        [-1.1665e+00],
        [ 7.7525e-02],
        [-1.7523e+00],
        [ 2.8662e-02],
        [ 7.5243e-02],
        [ 3.4550e-01],
        [-1.0415e-01],
        [-2.6029e-01],
        [-1.6750e+00],
        [ 1.1488e+00],
        [ 1.5835e+00],
        [-1.3424e+00],
 

In [12]:
# Task 5. Train GAN
"""
Function that trains a GAN
"""
def train_gan(learning_rate=1e-3,
              batch_size=512,
              steps=5000,
              discriminator_steps=20,
              generator_steps=20,
              generator_input=1,
              generator_hidden=5,
              generator_output=1,
              discriminator_input=1,
              discriminator_hidden=10,
              discriminator_output=1):
    """
    Learning rate = 1e-3
    Batch size = 512
    Number of Iterations = 5000
    Number of steps for descriminator and generator = 20
    Use both Discriminator and Generator classes inside of function
    Returns the fake distribution of the Generator of type torch.Tensor()
    """
    generator = Generator(generator_input,
                          generator_hidden,
                          generator_output)
    discriminator = Discriminator(discriminator_input,
                                  discriminator_hidden,
                                  discriminator_output)
    generator_optimizer = optim.SGD(generator.parameters(), learning_rate)
    discriminator_optimizer = optim.SGD(discriminator.parameters(),
                                        learning_rate)
    loss = torch.nn.BCELoss()
    for step in range(steps):
        if step % 1000 == 0:
            print("Reached step {}".format(step))
        train_dis(generator, discriminator, generator_input, discriminator_input, batch_size, 1, discriminator_optimizer, loss)
        train_gen(generator, discriminator, generator_input, batch_size, 1, generator_optimizer, loss)

    return generator(sample_Z(mu=0, sigma=1, sampleType="G", size=(batch_size, generator_input)))

In [14]:
# 5-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.express as px

ganTrainer = train_gan

fakeData = ganTrainer()

values = fakeData.data.storage().tolist()
fig = px.histogram(values, title="Histogram of Forged Distribution",nbins=50)
fig.show()

Reached step 0
Reached step 1000
Reached step 2000
Reached step 3000
Reached step 4000
